In [2]:
import altair as alt
from pathlib import Path
import json
import pandas as pd
import numpy as np

alt.renderers.enable('altair_viewer', inline=True)

RendererRegistry.enable('altair_viewer')

# Process Data

In [39]:
results = {
    'no-static': ('../results/17_09_51-08_2020/data.csv', '../results/17_09_51-08_2020/sim.csv'),
    'static': ('../results/14_15_10-08_2020/data.csv', '../results/14_15_10-08_2020/sim2.csv')
}
data_lib = {}

for k, v in results.items():
    futil_hls_data = pd.read_csv(v[0])
    futil_latency = pd.read_csv(v[1])
    futil_hls_data = futil_hls_data[futil_hls_data['type'] != 'hls_total']
    for _, row in futil_latency.iterrows():
        bench_filt = futil_hls_data['benchmark'] == row[0]
        type_filt = futil_hls_data['type'] == 'futil'
        futil_hls_data.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
    futil_hls_data = futil_hls_data.sort_values(by='benchmark').reset_index(drop=True)
    data_lib[k] = futil_hls_data

# merge futil latency data with the main dataframe
# futil_hls_data = pd.read_csv('../results/14_15_10-08_2020/data.csv')
# futil_latency = pd.read_csv('../results/14_15_10-08_2020/sim.csv')
# futil_hls_data = futil_hls_data[futil_hls_data['type'] != 'hls_total']
# for _, row in futil_latency.iterrows():
#     bench_filt = futil_hls_data['benchmark'] == row[0]
#     type_filt = futil_hls_data['type'] == 'futil'
#     futil_hls_data.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
# futil_hls_data = futil_hls_data.sort_values(by='benchmark').reset_index(drop=True)
# futil_hls_data
data_lib

{'no-static':                   benchmark   type   lut  dsp  latency  meet_timing  \
 0        linear-algebra-2mm    hls   939   12   3718.0            1   
 1        linear-algebra-2mm  futil  4039   12  26358.0            1   
 2        linear-algebra-3mm    hls   742    9   5043.0            1   
 3        linear-algebra-3mm  futil   651    9  28457.0            1   
 4       linear-algebra-atax  futil  1174    6   3601.0            1   
 5       linear-algebra-atax    hls   610    6    715.0            1   
 6       linear-algebra-bicg  futil   377    6   1940.0            1   
 7       linear-algebra-bicg    hls   327    6    218.0            1   
 8   linear-algebra-cholesky  futil   604    6   3138.0            1   
 9   linear-algebra-cholesky    hls  4040    6   1385.0            1   
 10    linear-algebra-durbin    hls  1070   12    366.0            1   
 11    linear-algebra-durbin  futil   958   12   1391.0            1   
 12      linear-algebra-gemm  futil  2690    9  144

# Timed Futil vs VHLS Graph

In [45]:
df = data_lib['static']
seq = pd.read_csv('../small_seq.csv')

data = pd.DataFrame()
for bench in df['benchmark'].unique():
    rows = df[df['benchmark'] == bench]
    hls = rows[rows['type'] == 'hls']
    futil = rows[rows['type'] == 'futil']
    b = bench.split('-')[2]
    data = data.append({
        'benchmark': b,
        'type': f"lut",
        'value': futil['lut'].values[0] / hls['lut'].values[0],
        'meet_timing': futil['meet_timing'].values[0],
        'equal': 1
    }, ignore_index=True)
#     data = data.append({
#         'benchmark': bench,
#         'type': f"dsp",
#         'value': futil['dsp'].values[0] / hls['dsp'].values[0],
#         'meet_timing': futil['meet_timing'].values[0],
#         'equal': 1
#     }, ignore_index=True)
    data = data.append({
        'benchmark': b,
        'type': "latency",
        'value': futil['latency'].values[0] / hls['latency'].values[0],
        'meet_timing': futil['meet_timing'].values[0],
        'equal': 1
    }, ignore_index=True)
#     data = data.append({
#         'benchmark': b,
#         'type': "lseq",
#         'value': seq.loc[seq['benchmark'] == bench,'latency'].values[0] / hls['latency'].values[0],
#         'meet_timing': futil['meet_timing'].values[0],
#         'equal': 1
#     }, ignore_index=True)

data['lut_mean'] = data.loc[data['type'] == 'lut', 'value'].mean()
data['latency_mean'] = data.loc[data['type'] == 'latency', 'value'].mean()

bars = alt.Chart().mark_bar().encode(
    x=alt.X('type', axis=alt.Axis(offset=2.0, labels=False, title=None)),
    y=alt.Y('value',
            scale=alt.Scale(type='log'),
            axis=alt.Axis(title="VHLS / Calyx", grid=False)),
    tooltip=['value', 'latency_mean', 'lut_mean'],
    color=alt.condition('datum.meet_timing == 0', alt.ColorValue('red'), 'type'),
)

vline = alt.Chart().mark_rule(color='red', size=0.5).encode(y='equal')
# meanLutLine = alt.Chart().mark_rule(color='blue').encode(y='lut_mean')
# meanLatLine = alt.Chart().mark_rule(color='green').encode(y='latency_mean')
height=300
alt.layer(bars, vline, data=data).facet(
    column=alt.Column('benchmark', header=alt.Header(labelOrient='bottom', labelAngle=20, labelPadding=10, labelAnchor='start')),
    spacing=1,
).configure_title(
    orient='bottom'
).configure_view(
    continuousHeight=height
).configure_bar(
    y2=height/2
)

alt.FacetChart(...)

# HLS Static vs No-static

In [54]:
static = data_lib['static']
no_static = data_lib['no-static']
df = pd.concat([static, no_static], keys=['static', 'no-static']).reset_index().drop(['level_1'], axis='columns').rename({'level_0': 'timing'}, axis='columns')
alt.Chart(df[df['type'] == 'futil']).mark_bar().encode(
    x='timing',
    y='lut',
    column=alt.Column('benchmark', header=alt.Header(labelAngle=-20)),
    color='timing'
)

alt.Chart(...)

# Systolic Array Data

In [138]:
systolic_data = pd.read_csv('../systolic-experiments/scan/data.csv')
systolic_lat = pd.read_csv('../systolic-experiments/scan/systolic.csv')
hls = pd.read_csv('../systolic-experiments/scan/hls.csv')
hls = hls[hls['type'] == 'hls'].sort_values(by='benchmark').reset_index(drop=True)
systolic_data = systolic_data[systolic_data['type'] == 'futil'].sort_values(by='benchmark').drop(columns=['meet_timing', 'source', 'type']).reset_index(drop=True)
systolic_data['latency'] = systolic_lat['latency']
# systolic_data['hls_lat'] = hls['latency']
# systolic_data['hls_dsp'] = hls['dsp']
# systolic_data['hls_lut'] = hls['lut']
systolic_data['lut / 10'] = systolic_data['lut'] / 10
d = systolic_data.melt(['benchmark'])
alt.Chart(d).mark_bar().encode(
    x='variable',
    y='value',
    color='variable',
    column=alt.Column('benchmark', header=alt.Header(labelAngle=-20)),
    tooltip=['value']
).transform_filter(
    'datum.variable != "lut"'
)

alt.Chart(...)